In [1]:
%pylab inline
path="./Bike-Sharing-Dataset/hour_noheader.csv"
raw_data=sc.textFile(path)
raw_data.first()

Populating the interactive namespace from numpy and matplotlib


'1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0,3,13,16'

In [2]:
raw_data.count()
raw_data.cache()

./Bike-Sharing-Dataset/hour_noheader.csv MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:-2

In [3]:
records=raw_data.map(lambda t:t.split(","))
first=records.first() #first는 records의 첫번째 데이터를 의미
print(first)
num_data=records.count() # num_data는 총 데이터의 수가 얼마인지 나타낸다.
print(num_data)

['1', '2011-01-01', '1', '0', '1', '0', '0', '6', '0', '1', '0.24', '0.2879', '0.81', '0', '3', '13', '16']
17379


In [4]:
records.cache()
records.take(3)

[['1',
  '2011-01-01',
  '1',
  '0',
  '1',
  '0',
  '0',
  '6',
  '0',
  '1',
  '0.24',
  '0.2879',
  '0.81',
  '0',
  '3',
  '13',
  '16'],
 ['2',
  '2011-01-01',
  '1',
  '0',
  '1',
  '1',
  '0',
  '6',
  '0',
  '1',
  '0.22',
  '0.2727',
  '0.8',
  '0',
  '8',
  '32',
  '40'],
 ['3',
  '2011-01-01',
  '1',
  '0',
  '1',
  '2',
  '0',
  '6',
  '0',
  '1',
  '0.22',
  '0.2727',
  '0.8',
  '0',
  '5',
  '27',
  '32']]

In [5]:
def get_mapping(rdd, idx):
    return rdd.map(lambda f:f[idx]).distinct().zipWithIndex().collectAsMap();

In [6]:
mappings=[get_mapping(records, i) for i in range(2,10)] # List comprehension
# 3번째부터 10번째 특징까지 카테고리 특징을 바이너리 카테고리로 변형시킨다.
print("Mapping of first cat features colums: %s"%get_mapping(records, 2))

Mapping of first cat features colums: {'4': 0, '1': 1, '3': 2, '2': 3}


In [7]:
for i in mappings: # 각 카테고리별 인덱스 
    print(i)

{'4': 0, '1': 1, '3': 2, '2': 3}
{'0': 0, '1': 1}
{'10': 0, '4': 3, '5': 6, '3': 7, '2': 8, '7': 9, '1': 5, '12': 1, '8': 2, '9': 4, '11': 10, '6': 11}
{'10': 0, '18': 14, '17': 2, '5': 15, '13': 20, '7': 19, '0': 4, '14': 9, '22': 3, '16': 11, '11': 21, '19': 13, '15': 23, '12': 1, '3': 17, '2': 18, '1': 5, '6': 22, '4': 6, '23': 16, '9': 12, '21': 10, '20': 7, '8': 8}
{'0': 0, '1': 1}
{'4': 0, '1': 1, '5': 4, '3': 5, '2': 6, '0': 2, '6': 3}
{'0': 0, '1': 1}
{'4': 0, '1': 1, '3': 2, '2': 3}


In [8]:
print(type(mappings))
print(type(records.first()))
cat_len=sum(list(map(len, mappings)))
num_len=len(records.first()[11:15]) #12,15 번째 float 특징은 변환없이 그대로 사용
total_len=cat_len+num_len

<class 'list'>
<class 'list'>


In [9]:
print("cat : %d\nnum : %d\ntotal : %d"%(cat_len, num_len, total_len))

cat : 57
num : 4
total : 61


In [10]:
from pyspark.mllib.regression import LabeledPoint
import numpy as np

In [11]:
def extract_features(record):
    cat_vec=np.zeros(cat_len)
    i=0
    step=0
    for field in record[2:9]: # 각 레코드별로 filed로 치환
        m=mappings[i] #i 값과 filed와 동률로 증가해야함
        idx=m[field] # i는 실제 0값을 표기할 위치에 대한 매핑
        cat_vec[idx+step]=1 # filed는 record를 넘기기 위한 매핑
        i=i+1
        step+=len(m)
    
    num_vec=np.array([float(field) for field in record[10:14]])
    return np.concatenate(cat_vec, num_vec)

def extract_label(record):
    return float(record[-1])

In [12]:
records[2]

TypeError: 'PipelinedRDD' object does not support indexing